# Workplace for: Customer: 1; DFU: Рис круглозерный 500 гр

In [41]:
import plotly.express as px
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from lightgbm import LGBMRegressor

In [42]:
df = pd.read_csv("data_grouped/train_group_1_Рис круглозерный 500 гр.csv")
sales_2_df_t = pd.read_csv("data_test_grouped/test_group_1_Рис круглозерный 500 гр.csv")
df, sales_2_df_t

(                         DFU  Customer      Period  BPV  Total Sell-in  \
 0    Рис круглозерный 500 гр         1  2022-01-24  NaN            NaN   
 1    Рис круглозерный 500 гр         1  2022-01-31  NaN            NaN   
 2    Рис круглозерный 500 гр         1  2022-02-07  NaN            NaN   
 3    Рис круглозерный 500 гр         1  2022-02-14  NaN            NaN   
 4    Рис круглозерный 500 гр         1  2022-02-21  NaN            NaN   
 ..                       ...       ...         ...  ...            ...   
 240  Рис круглозерный 500 гр         1  2021-12-20  NaN            NaN   
 241  Рис круглозерный 500 гр         1  2021-12-27  NaN            NaN   
 242  Рис круглозерный 500 гр         1  2022-01-03  NaN            NaN   
 243  Рис круглозерный 500 гр         1  2022-01-10  NaN            NaN   
 244  Рис круглозерный 500 гр         1  2022-01-17  NaN            NaN   
 
      Season  Type Geography End of Period  BPV_sale_period  SoD_sale_period  
 0    Winter  Сеть 

## Visualization

In [43]:
unique_stat_colors = {
    'BPV Mean': '#1f77b4',
    'BPV Median': '#ff7f0e',
    'BPV CI Low (95%)': '#2ca02c',
    'BPV CI Up (95%)': '#d62728',
    'Total Sell-in Mean': '#9467bd',
    'Total Sell-in Median': '#8c564b',
    'Total Sell-in CI Low (95%)': '#e377c2',
    'Total Sell-in CI Up (95%)': '#7f7f7f',
}

for (dfu, customer), group in df.groupby(['DFU', 'Customer']):
    melted_group = group.melt(id_vars=['Period'], value_vars=['BPV', 'Total Sell-in'],
                              var_name='Metric', value_name='Value')

    stat_lines = []

    for metric in ['BPV', 'Total Sell-in']:
        data = group[metric].dropna()
        if not data.empty:
            mean_val = data.mean()
            median_val = data.median()
            std_val = data.std()

            stat_lines.extend([
                (metric, f'{metric} Mean: {mean_val:.2f}', mean_val, unique_stat_colors[f'{metric} Mean']),
                (metric, f'{metric} Median: {median_val:.2f}', median_val, unique_stat_colors[f'{metric} Median'])
            ])

    fig = px.line(
        melted_group,
        x='Period', y='Value', color='Metric',
        title=f'DFU: {dfu} | Customer: {customer}',
        labels={'Value': 'Sales', 'Period': 'Date'}
    )

    for metric, label, value, color in stat_lines:
        fig.add_trace(go.Scatter(
            x=[group['Period'].min(), group['Period'].max()],
            y=[value, value],
            mode='lines',
            name=label,
            line=dict(dash='dot', color=color, width=1.5),
            showlegend=True
        ))

    for metric in ['BPV', 'Total Sell-in']:
        metric_df = group[['Period', metric]].dropna()
        if not metric_df.empty:
            metric_df = metric_df.copy()
            metric_df['Period_ordinal'] = pd.to_datetime(metric_df['Period']).map(pd.Timestamp.toordinal)
            X = metric_df[['Period_ordinal']]
            y = metric_df[metric]
            model = LinearRegression().fit(X, y)
            y_pred = model.predict(X)

            fig.add_trace(go.Scatter(
                x=metric_df['Period'],
                y=y_pred,
                mode='lines',
                name=f'Trend - {metric}',
                line=dict(dash='dash', color='orange')
            ))

    if group[['BPV', 'Total Sell-in']].dropna().shape[0] > 1:
        corr = group['BPV'].corr(group['Total Sell-in'])
        fig.add_annotation(
            text=f"Корреляция BPV и Total Sell-in: {corr:.2f}",
            xref="paper", yref="paper",
            x=0.99, y=1.05, showarrow=False,
            font=dict(size=12, color="white"),
            align="right",
            bordercolor="white", borderwidth=1
        )

    filtered_sales_2 = sales_2_df_t[(sales_2_df_t['DFU'] == dfu) & (sales_2_df_t['Customer'] == customer)]
    if not filtered_sales_2.empty:
        melted_sales_2 = filtered_sales_2.melt(id_vars=['Period'], value_vars=['BPV', 'Total Sell-in'],
                                               var_name='Metric', value_name='Value')
        for metric in melted_sales_2['Metric'].unique():
            metric_data = melted_sales_2[melted_sales_2['Metric'] == metric]
            color = 'black' if metric == 'BPV' else 'gray'
            fig.add_trace(go.Scatter(
                x=metric_data['Period'],
                y=metric_data['Value'],
                mode='lines+markers',
                name=f'test_sales - {metric}',
                line=dict(color=color, width=2, dash='dot')
            ))

    fig.update_xaxes(title_text='Дата', tickformat='%Y-%m-%d')
    fig.update_yaxes(title_text='Значение')
    fig.show()


## Machine learning

In [44]:
def compute_metrics(y_true, y_pred, m=1):
    """
    Compute forecast accuracy metrics: WAPE, MAE, RMSE, MASE.

    Parameters
    ----------
    y_true : array-like of shape (n,)
        Истинные значения.
    y_pred : array-like of shape (n,)
        Предсказанные значения.
    m : int, default=1
        Шаг «наивного» прогноза для расчёта MASE.
        Для несезонного ряда обычно m=1;
        для сезонного — период сезонности.

    Returns
    -------
    dict
        {
            'WAPE': float,  # в процентах
            'MAE': float,
            'RMSE': float,
            'MASE': float
        }
    """
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    n = len(y_true)

    # WAPE
    num = np.abs(y_true - y_pred).sum()
    denom = np.abs(y_true).sum()
    wape = num / denom * 100 if denom != 0 else np.nan

    # MAE
    mae = np.mean(np.abs(y_true - y_pred))

    # RMSE
    rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))

    # MASE
    if n > m:
        naive_errors = np.abs(y_true[m:] - y_true[:-m])
        scale = np.mean(naive_errors)
        mase = mae / scale if scale != 0 else np.nan
    else:
        mase = np.nan

    return {
        'WAPE': wape,
        'MAE': mae,
        'RMSE': rmse,
        'MASE': mase
    }

### Загрузка данных для товара "Рис круглозерный 500 гр"

- Загружаются обучающая и тестовая выборки для соответствующей товарной группы.
- Обучающая выборка фильтруется по датам: остаются только записи с 24 августа 2018 года по 17 января 2022 года.
- В тестовой выборке столбец `BPV` переименовывается в `BPV_true` для согласованности с последующей обработкой и оценкой качества прогноза.

In [45]:
train_path = "data_grouped/train_group_1_Рис круглозерный 500 гр.csv"
test_path = "data_test_grouped/test_group_1_Рис круглозерный 500 гр.csv"

train_full = pd.read_csv(train_path)
train_full = train_full[
    (train_full["Period"] >= "2018-08-24") &
    (train_full["Period"] <= "2022-01-17")
    ].copy()

test = pd.read_csv(test_path)
test = test.rename(columns={"BPV": "BPV_true"})

### Преобразование дат и создание числового временного признака

- Преобразуем столбцы `Period` в формат `datetime` в обеих выборках для корректной работы с датами.
- Добавляем признак `Period_ord` — порядковый номер даты (количество дней от фиксированной точки во времени).
  Этот числовой признак будет использоваться как входной параметр для модели, позволяя учитывать временную структуру данных.

In [46]:
train_full["Period"] = pd.to_datetime(train_full["Period"])
test["Period"] = pd.to_datetime(test["Period"])

train_full["Period_ord"] = train_full["Period"].map(pd.Timestamp.toordinal)
test["Period_ord"] = test["Period"].map(pd.Timestamp.toordinal)

### Добавление признака продаж за аналогичный период прошлого года

- Рассчитывается дата аналогичной недели прошлого года (`Period_last_year`) путём вычитания 52 недель от текущей даты.
- Формируется вспомогательная таблица `sellin_map`, содержащая пары: дата (`Period_last_year`) и продажи (`sellin_last_year`), полученные из обучающей выборки.
- Выполняется объединение основной обучающей и тестовой таблиц с `sellin_map` по колонке `Period_last_year`.
- В результате добавляется признак `sellin_last_year`, отражающий объём продаж в соответствующем периоде предыдущего года.
- Пропущенные значения заполняются нулями в случае отсутствия данных за аналогичный период.

In [47]:
train_full["Period_last_year"] = train_full["Period"] - pd.DateOffset(weeks=52)
test["Period_last_year"] = test["Period"] - pd.DateOffset(weeks=52)

sellin_map = train_full[["Period", "Total Sell-in"]].copy()
sellin_map = sellin_map.rename(columns={
    "Period": "Period_last_year",
    "Total Sell-in": "sellin_last_year"
})

train_full = train_full.merge(
    sellin_map,
    on="Period_last_year",
    how="left"
)
train_full["sellin_last_year"] = train_full["sellin_last_year"].fillna(0)

test = test.merge(
    sellin_map,
    on="Period_last_year",
    how="left"
)
test["sellin_last_year"] = test["sellin_last_year"].fillna(0)

### Создание дополнительных признаков для обучения модели

- `bpv_vs_sod_flag` — бинарный признак: равен 1, если продажи BPV превышают продажи SoD в текущем периоде.
- `bpv_minus_sod` — абсолютная разница между значениями `BPV_sale_period` и `SoD_sale_period`.
- `bpv_div_sod` — отношение BPV к SoD; добавление малого значения в знаменатель предотвращает деление на ноль.
- `weekofyear` — номер недели в году, отражающий возможные сезонные закономерности.
- `month` — номер месяца, также используется для захвата сезонных паттернов в данных.

In [48]:
train_full["bpv_vs_sod_flag"] = (
        train_full["BPV_sale_period"] > train_full["SoD_sale_period"]
).astype(int)

train_full["bpv_minus_sod"] = (
        train_full["BPV_sale_period"] - train_full["SoD_sale_period"]
)

train_full["bpv_div_sod"] = (
        train_full["BPV_sale_period"] / (train_full["SoD_sale_period"] + 1e-9)
)

train_full["weekofyear"] = train_full["Period"].dt.isocalendar().week
train_full["month"] = train_full["Period"].dt.month

### Подготовка данных к обучению модели и прогнозированию

- `pred_dates` — сохраняются даты, для которых отсутствует значение `BPV` и требуется сделать прогноз.
- Удаляются лишние столбцы (`Customer`, `DFU`, `Period`, `Period_last_year`), не участвующие в моделировании.
- `feature_cols` — список признаков, используемых для обучения модели. Включают временные характеристики и относительные метрики между `BPV_sale_period` и `SoD_sale_period`.
- `train_df` — выборка с известными значениями `BPV`, используется для обучения.
- `X_train` и `y_train` — матрица признаков и целевая переменная для обучения модели.
- `pred_df` — строки с отсутствующими значениями `BPV`, предназначенные для предсказания.
- `X_pred` — матрица признаков для прогнозирования.
- `pred_bpv_sale` — сохраняются значения `BPV_sale_period` для последующей постобработки предсказаний.

In [49]:
pred_dates = train_full.loc[train_full["BPV"].isna(), "Period"]

drop_cols = ["Customer", "DFU", "Period", "Period_last_year"]
train_full = train_full.drop(columns=[c for c in drop_cols if c in train_full.columns])

feature_cols = [
    "Period_ord",
    "BPV_sale_period",
    "SoD_sale_period",
    "bpv_vs_sod_flag",
    "bpv_minus_sod",
    "bpv_div_sod",
    "weekofyear",
]

train_df = train_full[train_full["BPV"].notna()].reset_index(drop=True)
X_train = train_df[feature_cols]
y_train = train_df["BPV"].values

pred_df = train_full[train_full["BPV"].isna()].reset_index(drop=True)
X_pred = pred_df[feature_cols]
pred_bpv_sale = pred_df["BPV_sale_period"]

### Обучение модели градиентного бустинга

- Инициализируется модель `LGBMRegressor` с фиксированным параметром `random_state=42` для воспроизводимости результатов.
- Модель обучается на обучающей выборке `X_train` с использованием целевой переменной `y_train`, представляющей значения BPV.

In [50]:
model = LGBMRegressor(random_state=42)
model.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000162 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 130
[LightGBM] [Info] Number of data points in the train set: 148, number of used features: 7
[LightGBM] [Info] Start training from score 21.039588
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -i

LGBMRegressor(random_state=42)

### Генерация и постобработка предсказаний

- `y_pred_full` — предсказанные моделью значения `BPV` для периодов, в которых отсутствовали реальные значения.
- Если в исходных данных значение `BPV_sale_period` равно нулю, соответствующие предсказания принудительно обнуляются — это отражает отсутствие продаж в канале.
- Все предсказания ограничиваются снизу нулём с помощью `np.maximum`, чтобы избежать появления отрицательных значений BPV.

In [51]:
y_pred_full = model.predict(X_pred)

y_pred_full = np.where(pred_bpv_sale == 0, 0, y_pred_full)

y_pred_full = np.maximum(y_pred_full, 0)

### Оценка качества прогноза

- Формируется датафрейм `merged`, содержащий:
  - `Period` — даты, для которых были сделаны предсказания,
  - `BPV_pred` — предсказанные моделью значения.
- Объединение с тестовой выборкой по дате позволяет добавить реальные значения `BPV_true`.
- Пропущенные значения в `BPV_true` заполняются нулями, что может отражать периоды с нулевыми продажами в тесте.
- С помощью функции `compute_metrics` рассчитываются основные метрики качества прогноза:
  - `WAPE` — взвешенная средняя абсолютная ошибка,
  - `MAE` — средняя абсолютная ошибка,
  - `RMSE` — среднеквадратичная ошибка,
  - `MASE` — масштабированная ошибка, учитывающая недельную сезонность (`m=1`).
- Метрики выводятся на экран для анализа эффективности модели.

In [52]:
merged = pd.DataFrame({
    "Period": pred_dates.reset_index(drop=True),
    "BPV_pred": y_pred_full
})

merged = merged.merge(
    test[["Period", "BPV_true"]],
    on="Period",
    how="left"
)

merged["BPV_true"] = merged["BPV_true"].fillna(0)

metrics = compute_metrics(
    merged["BPV_true"],
    merged["BPV_pred"],
    m=1
)

print(f"WAPE = {metrics['WAPE']:.2f}%")
print(f"MAE  = {metrics['MAE']:.4f}")
print(f"RMSE = {metrics['RMSE']:.4f}")
print(f"MASE = {metrics['MASE']:.4f}")

WAPE = 24.47%
MAE  = 5.2187
RMSE = 9.7507
MASE = 0.4658


In [53]:
to_save = merged.drop(columns=["BPV_true"])
to_save.to_csv("1_Рис круглозерный 500_гр_predictions.csv", index=False)

In [54]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=merged["Period"], y=merged["BPV_true"],
    mode="markers+lines",
    name="BPV_true",
    line=dict(color="blue"),
    marker=dict(size=6)
))
fig.add_trace(go.Scatter(
    x=merged["Period"], y=merged["BPV_pred"],
    mode="markers+lines",
    name="BPV_pred",
    line=dict(color="red", dash="dot"),
    marker=dict(size=6)
))
fig.update_layout(
    title="Сравнение BPV_true и BPV_pred (с обновленным набором признаков)",
    xaxis_title="Period",
    yaxis_title="BPV",
    legend_title="Легенда"
)
fig.show()